# 分析セクション
ここからは分析をするためのコードを書き出す。

### 月ごとの集計を始める

In [48]:
import pandas as pd

csv_file_path = 'test_modified.csv'
columns_to_read=['Date','SubjectCode','Amount']

df = pd.read_csv(csv_file_path,usecols=columns_to_read)

df['YearMonth']=df['Date'].str[:7]
df = df[['YearMonth','SubjectCode','Amount']]

pivot_df = df.pivot_table(index='YearMonth',columns='SubjectCode',values='Amount', aggfunc='sum').reset_index()
pivot_df = pivot_df.fillna(0)
pivot_df = pivot_df.astype({col: int for col in pivot_df.columns if col != 'YearMonth'})

pivot_df.to_csv('formatted_test.csv',index=False)

print(pivot_df)
#print(value: lambda monthly_summary['Subject'].values)

FileNotFoundError: [Errno 2] No such file or directory: 'test_modified.csv'

# 繰り越し

In [ ]:
import pandas as pd

csv_file_path = 'formatted_test.csv'
df = pd.read_csv(csv_file_path)


def calculate_balancs(df):
    """
    Calculate balance sheet
    """
    asset_columns = [col for col in df.columns if col.startswith('100')]
    liability_columns = [col for col in df.columns if col.startswith('200')]
    income_columns = [col for col in df.columns if col.startswith('400')]
    expense_columns = [col for col in df.columns if col.startswith('500')]
    df["TotalAssets"]=df[asset_columns].sum(axis=1)
    df["TotalLiabilities"]=df[liability_columns].sum(axis=1)
    df["TotalIncome"]=df[income_columns].sum(axis=1)
    df["TotalExpenses"]=df[expense_columns].sum(axis=1)
    df["NetIncome"]= df['TotalIncome']-df['TotalExpenses']
    df["TotalEquity"]= df['TotalAssets']-df['TotalLiabilities']

    for i in range(1,len(df)):
        df.at[i, 'TotalAssets'] += df.at[i-1, 'TotalAssets']
        df.at[i, 'TotalLiabilities'] += df.at[i-1, 'TotalLiabilities']
        df.at[i, 'TotalIncome'] += df.at[i-1, 'TotalIncome']
        df.at[i, 'TotalExpenses'] += df.at[i-1, 'TotalExpenses']
        df.at[i, 'NetIncome'] += df.at[i-1, 'NetIncome']
        df.at[i, 'TotalEquity'] += df.at[i-1, 'TotalEquity']

    return df
result_df = calculate_balancs(df)
print(df)


  YearMonth   100    101    102    130    200    300    400   490   500  ...  \
0   2024-03 -6555 -26986  10000 -14100 -16856  52730 -43250 -1320  4713  ...   
1   2024-04 -1410  49114  15000      0      0      0 -71194     0     0  ...   

     572    590   599    600  TotalAssets  TotalLiabilities  TotalIncome  \
0  16200  14000  1820 -52730        -6555            -16856       -43250   
1      0      0     0      0        -7965            -16856      -114444   

   TotalExpenses  NetIncome  TotalEquity  
0           4713     -47963        10301  
1           4713    -119157         8891  

[2 rows x 28 columns]


In [73]:
import pandas as pd

class BalanceSheet:
    def __init__(self, csv_file_path):
        """
        Initialize BalanceSheet object.

        Args:
            csv_file_path (str): Path to the CSV file containing the data.
        """
        self.csv_file_path = csv_file_path
        self.df = None

    def preprocess_and_pivot(self):
        """
        Preprocess the data read from the CSV file, and pivot and format the DataFrame.

        Args:

        Returns:
            None
        """
        # データの読み込み
        columns_to_read=['Date','SubjectCode','Amount']
        df = pd.read_csv(self.csv_file_path, usecols=columns_to_read)

        # データの前処理
        df['YearMonth'] = df['Date'].str[:7]
        df = df[['YearMonth','SubjectCode','Amount']]

        # ピボットとフォーマット
        pivot_df = df.pivot_table(index='YearMonth', columns='SubjectCode', values='Amount', aggfunc='sum').reset_index()
        pivot_df = pivot_df.fillna(0)
        pivot_df = pivot_df.astype({col: int for col in pivot_df.columns if col != 'YearMonth'})

        # self.dfに保存
        self.df = pivot_df

    def calculate_balances(self):
        """
        Calculate balances.

        Args:

        Returns:
            None
        """
        asset_columns = [col for col in self.df.columns if str(col).startswith('1')]
        liability_columns = [col for col in self.df.columns if str(col).startswith('2')]
        income_columns = [col for col in self.df.columns if str(col).startswith('4')]
        expense_columns = [col for col in self.df.columns if str(col).startswith('5')]

        self.df["TotalAssets"]=self.df[asset_columns].sum(axis=1)
        self.df["TotalLiabilities"]=self.df[liability_columns].sum(axis=1)
        self.df["TotalIncome"]=self.df[income_columns].sum(axis=1)
        self.df["TotalExpenses"]=self.df[expense_columns].sum(axis=1)
        self.df["NetIncome"]= self.df['TotalIncome']-self.df['TotalExpenses']
        self.df["TotalEquity"]= self.df['TotalAssets']-self.df['TotalLiabilities']

        for i in range(1,len(self.df)):
            self.df.at[i, 'TotalAssets'] += self.df.at[i-1, 'TotalAssets']
            self.df.at[i, 'TotalLiabilities'] += self.df.at[i-1, 'TotalLiabilities']
            self.df.at[i, 'TotalIncome'] += self.df.at[i-1, 'TotalIncome']
            self.df.at[i, 'TotalExpenses'] += self.df.at[i-1, 'TotalExpenses']
            self.df.at[i, 'NetIncome'] += self.df.at[i-1, 'NetIncome']
            self.df.at[i, 'TotalEquity'] += self.df.at[i-1, 'TotalEquity']

    def show_summary(self):
        """
        Print the summary DataFrame.

        Args:

        Returns:
            None
        """
        print(self.df)

    def save_summary(self, output_file_path):
        """
        Save the summary DataFrame to a CSV file.

        Args:
            output_file_path (str): Path to the output CSV file.

        Returns:
            None
        """
        self.df.to_csv(output_file_path, index=False)

    def make_balance_sheet(self, output_file_path):
        """
        Run all methods in the correct order to generate the balance sheet.

        Args:
            output_file_path (str): Path to the output CSV file.

        Returns:
            None
        """
        self.preprocess_and_pivot()
        self.calculate_balances()
        self.show_summary()
        self.save_summary(output_file_path)

# 使用例

input_file_path = 'datas/test_modified.csv'
# input_file_path = 'datas/financial_records_modified.csv'
output_file_path = 'datas/test_balance_sheet.csv'
# output_file_path = 'datas/financial_balance_sheet.csv'
bs = BalanceSheet(input_file_path)
bs.make_balance_sheet(output_file_path)


SubjectCode YearMonth   100    101    102    130    200    300    400   490  \
0             2024-03 -6555 -26986  10000 -14100 -16856  52730 -43250 -1320   
1             2024-04 -1410  49114  15000      0      0      0 -71194     0   

SubjectCode   500  ...    572    590   599    600  TotalAssets  \
0            4713  ...  16200  14000  1820 -52730       -37641   
1               0  ...      0      0     0      0        25063   

SubjectCode  TotalLiabilities  TotalIncome  TotalExpenses  NetIncome  \
0                      -16856       -44570          96326    -140896   
1                      -16856      -115764         104816    -220580   

SubjectCode  TotalEquity  
0                 -20785  
1                  41919  

[2 rows x 28 columns]


### 繰り越しを行う

In [5]:
import pandas as pd
import numpy as np
import datetime as dt

def get_subject_sum(df):
    """
    Get the sum of the each subject code.
    Args:
        dataframe: Dataframe containing 'SubjectCode', 'Amount'
            SubjectCode (str): Subject code.
            Amount (int): Account amount.
    Returns:
        dataframe: Dataframe containing 'SubjectCode', 'Amount', 'YearMonth'
    """
    yearmonths = df['YearMonth'].values
    yearmonths = np.unique(yearmonths)
    rows = []
    for yearmonth in yearmonths:
        item = df[df['YearMonth'] == yearmonth]
        subjects = item['SubjectCode'].unique()
        for subject in subjects:
            value = pd.to_numeric(item[item['SubjectCode'] == subject]['Amount'], errors='coerce').sum()
            row = {
                'YearMonth': yearmonth,
                'SubjectCode': subject,
                'Amount': value
            }
            rows.append(row)
    sums = pd.DataFrame(rows)
    sums = sums.sort_values(by=['YearMonth', 'SubjectCode'])
    return sums

def add_yearmonth_column(df):
    """
    日付列から年月を抽出して新しい列を追加する
    Returns:
        dataframe: Dataframe with 'Year' and 'Month' columns added
        yearmongh column added as string
    """
    df['Date']=pd.to_datetime(df['Date'])
    df['Year']=df['Date'].dt.year
    df['Month']=df['Date'].dt.month
    df['YearMonth'] = df['Date'].strftime('%Y-%m') #Seriesでエラーのためlambdaを使って適用する
    return df

df = pd.read_csv('tests/test.csv')
df = add_yearmonth_column(df)
sums = get_subject_sum(df)
print(df)

AttributeError: 'Series' object has no attribute 'strftime'

In [88]:
import pandas as pd
from datetime import datetime
from dateutil.relativedelta import relativedelta

input_file_path = 'datas/test_balance_sheet.csv'

df = pd.read_csv(input_file_path)
asset_columns = [col for col in df.columns if str(col).startswith('1')]
liability_columns = [col for col in df.columns if str(col).startswith('2')]

carryover_columns = asset_columns + liability_columns

months = df['YearMonth']
data = []

for month in months.values:
    item = df[df['YearMonth'] == month]
    for carryover_column in carryover_columns:
        carryover_value = item[carryover_column].values[0]
        formatted_month = month + "-01"
        date_obj = datetime.strptime(formatted_month, "%Y-%m-%d")

        # 1か月後の日付を計算
        date_obj += relativedelta(months=1)
        data.append({'Date': date_obj, 'SubjectCode': carryover_column, 'Amount': carryover_value, 'Remarks':"Carryover 99"})

new_df = pd.DataFrame(data, columns=['Date','ID', 'SubjectCode', 'Amount','Remarks'])
new_df= new_df.fillna('')

n_new_df = new_df.append(new_df)

print(n_new_df)

        Date ID SubjectCode  Amount       Remarks
0 2024-04-01            100   -6555  Carryover 99
1 2024-04-01            101  -26986  Carryover 99
2 2024-04-01            102   10000  Carryover 99
3 2024-04-01            130  -14100  Carryover 99
4 2024-04-01            200  -16856  Carryover 99
5 2024-05-01            100   -1410  Carryover 99
6 2024-05-01            101   49114  Carryover 99
7 2024-05-01            102   15000  Carryover 99
8 2024-05-01            130       0  Carryover 99
9 2024-05-01            200       0  Carryover 99
0 2024-04-01            100   -6555  Carryover 99
1 2024-04-01            101  -26986  Carryover 99
2 2024-04-01            102   10000  Carryover 99
3 2024-04-01            130  -14100  Carryover 99
4 2024-04-01            200  -16856  Carryover 99
5 2024-05-01            100   -1410  Carryover 99
6 2024-05-01            101   49114  Carryover 99
7 2024-05-01            102   15000  Carryover 99
8 2024-05-01            130       0  Carryover 99
